# Generate data

In [13]:
#from line_profiler import LineProfiler
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.stats import percentileofscore
import random
import numpy as np
import pandas as pd
import plotnine as gg
import torch
import torch.optim as optim
#from dataloader import TabularDataset
from sklearn.ensemble import RandomForestClassifier

import higher

from torch import nn
# from acme.utils.loggers.terminal import TerminalLogger
import dataclasses
#import chex
#import haiku as hk
#import jax
#import jax.numpy as jnp
#import optax
import pandas as pd
#import warnings
import gpytorch
from torch.distributions.bernoulli import Bernoulli
#warnings.filterwarnings('ignore')
#import pipeline_var_l2_loss
import seaborn as sns
#from dataloader import TabularDataset
#from var_l2_loss_estimator import *
#from ENN import basenet_with_learnable_epinet_and_ensemble_prior
import matplotlib.pyplot as plt

#wandb.init()
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import torch.nn.init as init
from datetime import datetime
 
#data from https://drive.google.com/drive/u/1/folders/1WuXIzpYLrLNH6pn9zBMx6oRCCQPRz0F1

directory = '/shared/share_mala/data/eicu_train_test/'
train_csv = 'eicu_train_final.csv'
test_csv = 'eicu_test_final.csv'

df_train = pd.read_csv(directory + train_csv)
df_test = pd.read_csv(directory + test_csv)
df = pd.concat([df_train, df_test], axis = 0)
X_col = list(df.columns)[:-1]
Y = 'EVENT_LABEL'


#df = df.groupby('EVENT_LABEL', group_keys=False).apply(lambda x: x.sample(2000))
Y_data = np.array(df[Y])

'''
For real data -
Select 10 features - use the old file  — code we have - we need to change the code somewhat
Change the setting to a continuous one - use the 
Form the clusters - how many? - 51 clusters
Reduce the dataset size -  use the old file — on fly 

Train points - 100 - 500
Pool points - 1 cluster - 250 points |||||  50 clusters 5 points each 
Number of clusters - 1+50
Batch size to be acquired  - 5
Horizons - 5 horizons

Marginal distribution of x is not same as given dataset
Test points - 1 cluster - 20 points
                     25 clusters - 20 points
                       25 clusters  - 2 points
'''

'\nFor real data -\nSelect 10 features - use the old file  — code we have - we need to change the code somewhat\nChange the setting to a continuous one - use the \nForm the clusters - how many? - 51 clusters\nReduce the dataset size -  use the old file — on fly \n\nTrain points - 100 - 500\nPool points - 1 cluster - 250 points |||||  50 clusters 5 points each \nNumber of clusters - 1+50\nBatch size to be acquired  - 5\nHorizons - 5 horizons\n\nMarginal distribution of x is not same as given dataset\nTest points - 1 cluster - 20 points\n                     25 clusters - 20 points\n                       25 clusters  - 2 points\n'

In [2]:
model = RandomForestClassifier(
n_estimators=100,  # Number of trees in the forest
criterion='gini',  # Function to measure the quality of a split. Can also be 'entropy'.
max_depth=None,    # Maximum depth of the tree. If None, nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
min_samples_split=2,  # Minimum number of samples required to split an internal node
min_samples_leaf=1,   # Minimum number of samples required to be at a leaf node
bootstrap=True,       # Whether bootstrap samples are used when building trees
oob_score=False,      # Whether to use out-of-bag samples to estimate the generalization score
random_state=None,    # Controls both the randomness of the bootstrapping and the sampling of features to consider when looking for the best split
verbose=0,            # Controls the verbosity of the process
class_weight=None,    # Weights associated with classes. Can be 'balanced'.
)
    
def get_feat_importance(model, X_col): # get most important features, input is a randomforest classifier/regressor
    importances = model.feature_importances_
    forest_importances = pd.Series(importances, index = X_col)
    forest_importances = forest_importances.nlargest(10)  #only keep most important 10 feature
    feature_importances = list(forest_importances.index)
    return feature_importances #return the columns   


Y_train = df['EVENT_LABEL']
X_col = list(df.columns)
X_col.remove('EVENT_LABEL')
X_train = df[X_col]


model.fit(X_train, Y_train)
X_col = list(X_train.columns)
X_col_important = get_feat_importance(model, X_col)
# X_col_important.append('EVENT_LABEL')

X_train_imp = df[X_col_important]

Y_pred = model.predict_proba(X_train)
Y_pred = Y_pred[:,1]

In [9]:
# Normalize data
n_cluster = 30
data = np.array(X_train_imp)
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
kmeans = KMeans(n_clusters = n_cluster, random_state = 1)
kmeans.fit(data_normalized)

labels = kmeans.predict(data_normalized)

# Create a DataFrame
data = pd.DataFrame({'labels': labels, 'Y': Y_train})

# Calculate mean of Y for each unique value in X
result = data.groupby('labels')['Y'].agg(['mean', 'count'])

print(result)

/user/ym2865/.conda/envs/yuanzhe_new/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


            mean  count
labels                 
0       0.080972   3458
1       0.051369   2959
2       0.275904    830
3       0.032513   5198
4       0.232380   1433
5       0.044001   2909
6       0.679825    456
7       0.096234    239
8       0.050847   2360
9       0.273519    574
10      0.067797    354
11      0.071653   1591
12      0.313167   1124
13      0.231447    795
14      0.142857      7
15      0.016506   6967
16      0.027959   2754
17      0.187726    277
18      0.269113    654
19      0.051419   4687
20      0.854286    350
21      0.097585   1035
22      0.056099   1533
23      0.156275   1235
24      0.263208   1060
25      0.248982   1474
26      0.057462   3933
27      0.414634    205
28      0.108501   2341
29      0.201152   2257


# Select points from clusters

In [19]:

good_cluster_ind = 27
usual_cluster_ind = 3



n = df.shape[0]
labels = kmeans.labels_
centers = kmeans.cluster_centers_



def close_point_cluster(data, labels, centers, num_closest_points, desired_cluster_index):
    # Index of the desired cluster center
    X = data[labels == desired_cluster_index]

    desired_cluster_index = desired_cluster_index  # Change this to the desired cluster index

    # Calculate distances from each point to the desired cluster center
    distances_to_desired_center = np.linalg.norm(X - centers[desired_cluster_index], axis=1)

    # Sort the distances and select the indices of the 10 closest points
    closest_indices = np.argsort(distances_to_desired_center)[:num_closest_points]


    return closest_indices

def random_partition_list(lst, sizes):
    # Shuffle the list randomly
    random.shuffle(lst)
    
    # Partition the shuffled list into three parts with the specified sizes
    partitions = [lst[start:start+size] for start, size in zip([0, sizes[0], sizes[0] + sizes[1]], sizes)]
    
    return partitions

'''
ENN/Ensemble:
   2 clusters:
100 training points from 1 cluster
100 pool points from this useless cluster and 10 pool points from the other cluster
1 or 2 points - batch size
Test set - 20 points each 
Graphs: The setting (appendix), just compare it with random baselines, consistency results for multi clusters
Challenge: ENN - consistency is issue, Ensemble - slow (parallelization is necessary)
'''

good_cluster_indices = close_point_cluster(data_normalized, labels, centers, num_closest_points = 10 + 20, desired_cluster_index = good_cluster_ind)
usual_cluster_indices = close_point_cluster(data_normalized, labels, centers, num_closest_points = 100 +  100 + 20, desired_cluster_index = usual_cluster_ind)

[pool_indices_0, test_indices_0] = random_partition_list(good_cluster_indices, sizes = [10,20])
[train_indices, pool_indices_1, test_indices_1] = random_partition_list(usual_cluster_indices, sizes = [100, 100,20])

 # #train_indices
pool_indices_all = np.concatenate((pool_indices_0, pool_indices_1))
test_indices_all = np.concatenate((test_indices_0, test_indices_1)) 

print(len(train_indices), len(test_indices_all), len(pool_indices_all))

100 40 110


In [26]:
data_dir = '/shared/share_mala/data/eicu_train_test/0520_data_binary/'

index_list = [train_indices, pool_indices_all, test_indices_all]
name_list = ['train','pool','test']

for i in range(3):
    pd.DataFrame(data_normalized[index_list[i]]).to_csv(data_dir + name_list[i] + '_x.csv')
    pd.DataFrame(Y_train[index_list[i]]).to_csv(data_dir + name_list[i] + '_y.csv')